In [2]:
# IMPORT REQUIRED DEPENDENCIES
import os
from datetime import datetime

import tensorflow as tf

import models_config as models
import datasets_config as datasets
import feature_extractors_config as feature_extractors

In [14]:
# EXPERIMENT PARAMETERS
EXPERIMENT_NAME = 'mobilenet_experiments_1'
EXPERIMENT_PARAMS = [
    {'name': 'gru',
     'batch_size': 64,
     'epochs': 5,
     },
]

In [15]:
# DATA PARAMETERS
DATASETS_PARAMS = [
    {'dataset_path': '../datasets/UCF-3',
     'seq_len': 50,
     'train_test_split': .8
     },
]

In [16]:
# EXTRACTOR PARAMETERS
EXTRACTOR_PARAMS = [
    (feature_extractors.MobileNetV2Extractor, {}),
]

In [17]:
# MODEL PARAMETERS
MODEL_PARAMS = [
    (models.GRU.gru2, {'activation_function': 'relu',
                       'loss_function': 'sparse_categorical_crossentropy',
                       'optimizer': 'adam',
                       }),
]

In [18]:
# TRAIN MODEL AND SAVE TO saved_experiments DIRECTORY
def train_model(model, dataset, experiment_params):
    experiment_dir = os.path.join('../saved_experiments', EXPERIMENT_NAME)

    log_dir = os.path.join(experiment_dir, 'logs/fit/',
                           experiment_params['name'] + '_' + datetime.now().strftime("%Y%m%d-%H%M%S"))

    save_model_callback = tf.keras.callbacks.ModelCheckpoint(os.path.join(experiment_dir, experiment_params['name']),
                                                             monitor='val_loss',
                                                             verbose=1,
                                                             save_best_only=True,
                                                             options=None,)

    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir,
                                                          histogram_freq=1,
                                                          update_freq='epoch',)

    out = model.fit(
        dataset.train_dataset,
        validation_data=dataset.validation_dataset,
        epochs=experiment_params['epochs'],
        batch_size=experiment_params['batch_size'],
        callbacks=[tensorboard_callback, save_model_callback],
    )
    return out

In [19]:
if __name__ == '__main__':
    # train test loop
    for idx, (experiment_params, data_params, extractor_params, model_params) in \
            enumerate(zip(EXPERIMENT_PARAMS, DATASETS_PARAMS, EXTRACTOR_PARAMS, MODEL_PARAMS)):

        # init based on hyper parameters
        extractor = extractor_params[0](**extractor_params[1])  # get extractor
        dataset = datasets.Dataset.Training(**data_params, extractor=extractor)  # get data
        model = model_params[0](output_size=len(dataset.labels), **model_params[1])  # get model

        train_model(model, dataset, experiment_params)  # train model
        # test_model(model, dataset, experiment_params)  # evaluate model

Epoch 1/2
218/218 [==============================] - 39s 162ms/step - loss: 1.0129 - sparse_categorical_accuracy: 0.5311 - val_loss: 1.0270 - val_sparse_categorical_accuracy: 0.5234
Epoch 2/2
218/218 [==============================] - 38s 165ms/step - loss: 1.0077 - sparse_categorical_accuracy: 0.5335 - val_loss: 1.0210 - val_sparse_categorical_accuracy: 0.5248
